In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing list of stocks

Ideally, we would pay to connect directly to the index provider (Standard & Poor's) to pull their real-time constituents of the S&P 500 constituents. 

But we can find static versions of the S&P 500 constituesnt for free here.

In [2]:
stocks_full = pd.read_csv('sp_500_stocks.csv')
stocks = pd.DataFrame({'ticker': stocks_full['Symbol']})

In [24]:
stocks2 = stocks_full['Symbol']

## Acquiring an API token

We will import IEX Cloud API token to gather the financial data.

Using sandbox mode to gather randomized financial data instead of realtime data. This mode allows for testing functions properly before using the real API.

Note: need to restart the kernel in order to import the API token from secrets.py file at the same folder.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making the first API call

We will retrieve the following information:
* Market capitalization for each stock
* Price of each stock

In [8]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'

In [12]:
data = requests.get(api_url).json()
type(data)

dict

In [13]:
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'SSK/GAANTBCTG MRNQ DEL(LAL)S OAEE', 'calculationPrice': 'tops', 'open': None, 'openTime': None, 'openSource': 'cioalffi', 'close': None, 'closeTime': None, 'closeSource': 'aolficif', 'high': None, 'highTime': None, 'highSource': None, 'low': None, 'lowTime': None, 'lowSource': None, 'latestPrice': 141.664, 'latestSource': 'IEX real time price', 'latestTime': '2:33:41 PM', 'latestUpdate': 1635479582671, 'latestVolume': None, 'iexRealtimePrice': 141.018, 'iexRealtimeSize': 103, 'iexLastUpdated': 1678400588650, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 142.92, 'previousVolume': 126817799, 'change': -0.991, 'changePercent': -0.00732, 'volume': None, 'iexMarketPercent': 0.013137329237003286, 'iexVolume': 1225951, 'avgTotalVolume': 111693385

## Parsing API call

The API call that's executed in the last code block contains all the information required to build the equal-weight S&P 500 strategy. With that said, the data is not in a proper format yet and we need to parse it first.

In [16]:
price = data['latestPrice']
market_cap = data['marketCap']

market_cap/1000000000000

2.412738960475

## Adding stocks data to a Pandas DataFrame

Store all data in a tabular format initial trial.

In [22]:
colns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_df = pd.DataFrame(columns = colns)
final_df.append(
    pd.Series(
        [
            symbol,
            price,
            market_cap,
            'N/A'
        ],
        index = colns,
    ),
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,141.664,2412738960475,N/A


## Looping through the tickers in the list of stocks

In [30]:
final_df = pd.DataFrame(columns = colns)
for symbol in stocks2:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json() # single API calls
    final_df = final_df.append(
        pd.Series(
            [
                symbol,
                data['latestPrice'],
                data['marketCap'],
                'N/A'
            ],
            index = colns,
        ),
        ignore_index=True
    )

In [31]:
final_df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,MMM,181.580,100334939156,N/A
1,AOS,54.830,9080049937,N/A
2,ABT,110.110,198422806991,N/A
3,ABBV,108.600,185658791629,N/A
4,ABMD,325.364,14823866076,N/A
5,ACN,259.690,178219989284,N/A
6,ATVI,91.560,73849704041,N/A
7,ADBE,515.840,245143371452,N/A
8,AAP,160.910,10704832035,N/A
9,AMD,92.490,109751795818,N/A


## Using batch API calls to improve performance

HTTP requests are usually the slowest components of a script. API providers, on the other hand, will often provide discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls and we will split our list of stocks into groups of 100s for batch API calls.

In [32]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst"""
    for i in range(0, len(lst), n):
        yield lst[i:(i+n)]

In [50]:
symbol_groups = list(chunks(stocks2, 100))
symbol_strings = []

In [51]:
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])
final_df = pd.DataFrame(columns = colns)

In [52]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_df = final_df.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['marketCap'],
                    'N/A'
                ],
                index = colns,
            ),
            ignore_index=True
        )

final_df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,MMM,180.33,103818841128,N/A
1,AOS,55.52,9091424603,N/A
2,ABT,112.18,196124615330,N/A
3,ABBV,105.50,190053885195,N/A
4,ABMD,332.40,14797198132,N/A
5,ACN,261.16,176814879722,N/A
6,ATVI,93.31,71546843271,N/A
7,ADBE,504.79,250269745925,N/A
8,AAP,157.30,11100677330,N/A
9,AMD,92.24,111730864577,N/A


## Calculating the number of shares to buy

-- based on your portfolio

In [62]:
portfolio_size = input('Enter your portfolio:')

try:
    val = float(portfolio_size)
    print(val)
except ValueError:
    print("That's not a number! \nPlease try again:")
    portfolio_size = input('Enter your portfolio:')
    val = float(portfolio_size)

Enter your portfolio:1000000
1000000.0


In [63]:
position_size = val/len(final_df.index)

for i in range(0, len(final_df.index)):
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_df.loc[i, 'Stock Price'])

final_df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,MMM,180.33,103818841128,10
1,AOS,55.52,9091424603,35
2,ABT,112.18,196124615330,17
3,ABBV,105.50,190053885195,18
4,ABMD,332.40,14797198132,5
5,ACN,261.16,176814879722,7
6,ATVI,93.31,71546843271,21
7,ADBE,504.79,250269745925,3
8,AAP,157.30,11100677330,12
9,AMD,92.24,111730864577,21


## Formatting Excel output

In [72]:
# Initialize xlsxWriter object

writer = pd.ExcelWriter('computed_trades.xlsx', engine='xlsxwriter')
final_df.to_excel(writer, 'computed_trades', index=False)

In [73]:
# Create the formats needed for .xlsx file
# - String format for tickers
# - $xx.xx format for stock prices
# - $xx.xxx format for market capitalization
# - Integer format for the number of shares to purchase

background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [67]:
# Apply the formats to the columns of .xlsx file.

writer.sheets['computed_trades'].set_column('A:A', 18, string_format)
writer.sheets['computed_trades'].set_column('B:B', 18, dollar_format)
writer.sheets['computed_trades'].set_column('C:C', 18, dollar_format)
writer.sheets['computed_trades'].set_column('D:D', 18, integer_format)

# Handle column indices:

writer.sheets['computed_trades'].write('A1', 'Ticker', string_format)

# Save
writer.save()

In [74]:
# Simplify the previous code block by putting it in 2 loops:

column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['computed_trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['computed_trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

writer.save()